下準備

In [1]:
import osmnx as ox
import pickle
import os
import networkx as nx
import numpy as np
import time
import csv
import ujson
import ast

path = "c:/Users/owner/ACO"

道路ネットワークデータの準備

In [2]:
import osmnx as ox
import pickle

#osmnxのデータをデータベース化したい
# 対象地域の道路情報取得 (東京都調布市)
query = "Chofu,Tokyo,Japan"
city = query.split(',')[0].lower()
print(f"city:{city}")
try:
  # Pickleファイルからデータをロード
  with open(os.path.join(path,"chofu_map.pkl"), "rb") as f:
    G = pickle.load(f)
except Exception as e:
  # ファイルが無かったら作る
  G = ox.graph_from_place(query, network_type="walk") #query変えたら最初だけこっち
  # データをPickleファイルに保存
  with open("chofu_map.pkl", "wb") as f:
      pickle.dump(G, f)

# 道路グラフネットワーク可視化
fmap = ox.plot_graph_folium(G)
fmap.save(outfile="road_network.html")

opts = {"node_size": 5, "bgcolor": "white", "node_color": "blue", "edge_color": "blue"}
#ox.plot_graph(G, show=False, save=True, filepath="road_network.png", **opts)

city:chofu


C:\Users\owner\AppData\Local\Temp\ipykernel_7368\1986028127.py:21: UserWarning: The `folium` module has been deprecated and will be removed in a future release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  fmap = ox.plot_graph_folium(G)


スポットリストの処理

In [27]:
import pandas as pd

df = pd.read_csv(os.path.join(path,f"data/shaped_symbols_{city}.csv"))
df['node'] = df.apply(lambda row:ox.nearest_nodes(G,row['lon'],row['lat']),axis=1)
df['tags'] = df['tags'].apply(lambda x:ast.literal_eval(x))
df = df.sort_values(by=['node','name'],ascending=[True,True]).reset_index(drop=True)
# 'node'の値をグループ化して、'name'と'tags'を和でまとめる
df = df.groupby('node').agg({'lat':'first','lon':'first','name': lambda x: ', '.join(x), 'tags': 'sum'}).reset_index()
df = df.sort_values(by=['node','name'],ascending=[True,True]).reset_index(drop=True)
df['tags'] = df['tags'].apply(lambda x:str(x))
df = df.sort_values(by=['node','name'],ascending=[True,True]).reset_index(drop=True)
df.to_csv(os.path.join(path,f'data/nodes_symbols_{city}.csv'),index=False)
df.to_csv(os.path.join(path,f'data/shiftjis_nodes_symbols_{city}.csv'),encoding='shiftjis',index=False)
df_input = df[['lon','lat']]
df_input.to_csv(os.path.join(path,f'data/inputfile_{city}.csv'),index=False)
df_input.to_csv(os.path.join(path,f'data/shiftjis_inputfile_{city}.csv'),encoding='shiftjis',index=False)
df.tail()
df_input.tail()

,lon,lat
48,139.523392,35.660077
49,139.575498,35.658208
50,139.549382,35.646962
51,139.542222,35.664104
52,139.545057,35.655233


隣接行列を作る

In [13]:
with open(os.path.join(path,"chofu_map.pkl"), "rb") as f:
    H = pickle.load(f)
# 四捨五入された長さを持つ新しいグラフを作成
G = H.copy()
for u, v, data in G.edges(data=True):
    if 'length' in data:
        data['length'] = np.float64(round(data['length']))

In [14]:
with open(os.path.join(path,"chofu_map.pkl"), "rb") as f:
    H = pickle.load(f)
# 四捨五入された長さを持つ新しいグラフを作成
G = H.copy()
for u, v, data in G.edges(data=True):
    if 'length' in data:
        data['length'] = np.float64(round(data['length']))

# 保存する辞書型データ
index2node = {index:node for index,node in enumerate(G.nodes)}
node2index = {node:index for index,node in enumerate(G.nodes)}

# データをJSONファイルに書き込む
with open(os.path.join(path,"data/index2node.json"), "w") as json_file:
    ujson.dump(index2node, json_file)
with open(os.path.join(path,"data/node2index.json"), "w") as json_file:
    ujson.dump(node2index, json_file)

spot_nodes = pd.read_csv(os.path.join(path,f'data/nodes_symbols_{city}.csv'))
spot_list = pd.read_csv(os.path.join(path,f'data/nodes_symbols_{city}.csv'))['node'].tolist()
# spot_list = [n for index,n in enumerate(G.nodes()) if index%30==0]
# spot_list = sorted(spot_list)

timelist=[time.time()]
adj_mat = np.empty((0, len(spot_list)),dtype=np.float64)  # 初期化

for i,start_node in enumerate(spot_list):
    print(i,end=' ')
    # 最短距離を計算
    shortest_paths = nx.single_source_dijkstra_path_length(G, source=start_node, weight="length")
    sorted_shortest_paths = np.array([shortest_paths[key] for key in sorted(shortest_paths) if key in spot_list])
    if i==0:
        adj_mat = sorted_shortest_paths.copy()
    else:
        adj_mat = np.vstack((adj_mat,sorted_shortest_paths))
        
timelist.append(time.time())
print(f"\n実行時間:{int((timelist[-1]-timelist[-2])*1000)}ms")

# 配列をファイルに保存
np.save(os.path.join(path,'data/adjacent_matrix_%s.npy'%city), adj_mat)
# ファイルから配列を読み込む
loaded_array = np.load(os.path.join(path,'data/adjacent_matrix_%s.npy'%city))

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 
実行時間:7573ms


全てのノード情報を取得

In [68]:
# グラフのノードリストを取得
nodes = ox.graph_to_gdfs(G, edges=False)
# ノードの情報を表示
nodes["custom_index"] = range(len(nodes))
# "osmid"の代わりに"custom_index" 列を新しいインデックスとして設定
nodes["osmid"] = nodes.index
nodes.set_index("custom_index", inplace=True)

['{0: 0, 1: 1009, 2: 3360, 3: 4404, 4: 5847, 5: 4220, 6: 4346, 7: 4073, 8: 4432, 9: 2349, 10: 7178, 11: 4498, 12: 5210, 13: 4245, 14: 6507, 15: 1749, 16: 4404, 17: 2548, 18: 3647, 19: 2825, 20: 2578, 21: 3255, 22: 2819, 23: 1539, 24: 965, 25: 5166, 26: 2316, 27: 3043, 28: 4211, 29: 1843, 30: 303, 31: 6605, 32: 695, 33: 3074, 34: 2168, 35: 2134, 36: 6141, 37: 4388, 38: 3650, 39: 2403, 40: 1906, 41: 3499, 42: 3643, 43: 6257, 44: 297, 45: 2398, 46: 5342, 47: 1052, 48: 307, 49: 5340, 50: 3156, 51: 2275, 52: 2258}',
 '{0: 1009, 1: 0, 2: 2415, 3: 3459, 4: 4902, 5: 3382, 6: 3401, 7: 3107, 8: 3487, 9: 1406, 10: 6233, 11: 3532, 12: 4244, 13: 3279, 14: 5562, 15: 1046, 16: 3438, 17: 1710, 18: 2857, 19: 1859, 20: 1740, 21: 2417, 22: 1981, 23: 842, 24: 882, 25: 4221, 26: 1613, 27: 2205, 28: 3245, 29: 898, 30: 846, 31: 5660, 32: 1311, 33: 2129, 34: 1223, 35: 1189, 36: 5196, 37: 3443, 38: 2705, 39: 1565, 40: 1068, 41: 2661, 42: 2805, 43: 5312, 44: 876, 45: 1560, 46: 4397, 47: 285, 48: 951, 49: 4395, 

In [8]:
import os

file_path = os.path.join(path, 'data/adjacent_matrix_%s.npy' % city)  # ファイルのパスを取得

# ファイルの存在を確認してから容量を取得
if os.path.exists(file_path):
    file_size = os.path.getsize(file_path)
    file_size_in_kb = file_size / 1024  # ファイルサイズをKB単位に変換
    print(f"{file_size_in_kb} KB")
else:
    print("ファイルが見つかりません。")


661.6953125 KB


In [9]:
import re

# 数字を取り出す
def extract_chars(input_string):
    # 正規表現パターンを定義
    pattern = r'[0-9e.+-]'
    
    # パターンに一致する部分を取り出す
    result = re.findall(pattern, input_string)
    
    # 結果を文字列として連結する
    result_string = ''.join(result)
    
    return result_string

# テスト用の文字列
input_str = "abc123.45e-6xy+-z"

# 関数を呼び出して取り出した文字列を表示
output_str = extract_chars(input_str)
print(output_str)


123.45e-6+-


In [2]:
import numpy as np
import os

# ファイルのパスを指定
file_path = os.path.join('.', 'data/adjacent_matrix_chofu2.npy')

# ファイルを読み込む
if os.path.exists(file_path):
    adjacent_matrix_chofu2 = np.load(file_path)
    print(adjacent_matrix_chofu2)
else:
    print("ファイルが見つかりません。")

[[   0.     815.573 1349.514 ... 3890.798 1226.85  1142.678]
 [ 815.573    0.     785.116 ... 3783.493 1434.319 1616.05 ]
 [1349.514  785.116    0.    ... 3871.943 1522.769 2175.617]
 ...
 [3890.798 3783.493 3871.943 ...    0.    2835.436 4840.035]
 [1226.85  1434.319 1522.769 ... 2835.436    0.    2306.669]
 [1142.678 1616.05  2175.617 ... 4840.035 2306.669    0.   ]]
